In [2]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

Logging hadn't been started.


C:\Users\saheediyanda\Anaconda3\lib\site-packages\IPython\core\magics\logging.py:130: UserWarning: Couldn't start log: [Errno 2] No such file or directory: 'C:\\Users\\saheediyanda/.logs/dw.py'
  warn("Couldn't start log: %s" % sys.exc_info()[1])


In [3]:
from static_grader import grader

ModuleNotFoundError: No module named 'static_grader'

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np
import gzip

In [5]:
scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz', compression='gzip')
scripts.head()

FileNotFoundError: [Errno 2] No such file or directory: './dw-data/201701scripts_sample.csv.gz'

In [6]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', compression='gzip',names=col_names)
practices.head()

FileNotFoundError: [Errno 2] No such file or directory: './dw-data/practices.csv.gz'

In [7]:
chem = pd.read_csv('./dw-data/chem.csv.gz', compression='gzip',index_col=0)
chem.head()

FileNotFoundError: [Errno 2] No such file or directory: './dw-data/chem.csv.gz'

Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [10]:
def summary_func(df, elem):
    list_1 = df[elem]
    total = sum(list_1)
    mean = np.average(list_1)
    std = np.std(list_1)
    df2 = df.quantile(q=[.25,.5,.75])
    q25 = df2[elem][0.25]
    median = df2[elem][0.5]
    q75 = df2[elem][0.75]
    
    return (total, mean, std, q25, median, q75)
    
    
summary_stats = [
     ('items', summary_func(scripts, 'items')),
     ('quantity', summary_func(scripts, 'quantity')),
     ('nic', summary_func(scripts, 'nic')),
     ('act_cost', summary_func(scripts, 'act_cost'))]

In [11]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [12]:
group_by_bnf_name = scripts.groupby('bnf_name').sum().sort_values(['items'],ascending=0)
maxitem = group_by_bnf_name.iloc[0]
most_common_item = [(maxitem.name, maxitem['items'])]

In [13]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [14]:
code_post_code_min = pd.DataFrame(practices.groupby('code')['post_code'].min())
code_post_code_min.head()

,post_code
code,
A81001,TS18 1HU
A81002,TS18 2AW
A81003,TS25 1QU
A81004,TS1 3BE
A81005,TS14 7DJ


In [15]:
joined = scripts.merge(code_post_code_min, left_on='practice',right_index=True).sort_values('post_code').reset_index(drop=True)
joined.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code
0,Y02620,0407010H0,Paracet_Tab Solb 500mg,6,48.26,44.75,524,B11 4BW
1,M85078,1304000H0,Eumovate_Oint 0.05%,7,16.60,15.45,280,B11 4BW
2,M85078,1304000H0,Eumovate_Crm 0.05%,3,12.74,11.83,230,B11 4BW
3,M85078,1304000H0,Clobet But_Oint 0.05%,4,18.18,16.88,330,B11 4BW
4,M85078,1304000H0,Clobet But_Crm 0.05%,6,28.88,26.81,320,B11 4BW


In [16]:
# items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100

items_by_region_totals = joined.groupby(['post_code']).agg({'items': ['sum']}).reset_index().sort_values(by='post_code')

items_by_post_bnf_name = joined.groupby(['post_code','bnf_name'],as_index=False).agg({'items' : 'sum'})

items_by_region = items_by_post_bnf_name.groupby('post_code').apply(lambda row: row.sort_values(by='items',ascending=False).head(1)).reset_index(drop=True)
items_by_region['items'] = items_by_region['items'] / items_by_region_totals[('items', 'sum')]
tuples = [tuple(x) for x in items_by_region[:100].to_numpy()]


In [18]:
grader.score.dw__items_by_region(tuples)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [19]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [20]:
opioids_bnf_code = pd.DataFrame(chem[chem['NAME'].str.contains('|'.join(opioids),case=False)])
opioids_bnf_code.loc[:,'Opiod'] = 1
print(opioids_bnf_code.head())

                                          NAME  Opiod
CHEM SUB                                             
0104020D0  Codeine Phosphate Compound Mixtures      1
0104020N0                     Opium & Morphine      1
0309010C0                    Codeine Phosphate      1
0309010N0            Diamorphine Hydrochloride      1
0309010S0              Methadone Hydrochloride      1


In [21]:
joined_opiod = joined.merge(opioids_bnf_code['Opiod'], how="left",left_on = "bnf_code", right_index=True,sort=False).fillna(0)
joined_opiod.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,Opiod
0,Y02620,0407010H0,Paracet_Tab Solb 500mg,6,48.26,44.75,524,B11 4BW,0.0
1,M85078,1304000H0,Eumovate_Oint 0.05%,7,16.60,15.45,280,B11 4BW,0.0
2,M85078,1304000H0,Eumovate_Crm 0.05%,3,12.74,11.83,230,B11 4BW,0.0
3,M85078,1304000H0,Clobet But_Oint 0.05%,4,18.18,16.88,330,B11 4BW,0.0
4,M85078,1304000H0,Clobet But_Crm 0.05%,6,28.88,26.81,320,B11 4BW,0.0


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [22]:
summary_joined_opiod = joined_opiod.groupby('practice').agg({'bnf_name':'count','Opiod':['mean','std','sum']})
summary_joined_opiod.columns=summary_joined_opiod.columns.droplevel(0) #.rename(columns={"A": "a", "B": "c"})
opiod_mu = joined_opiod['Opiod'].mean()
summary_joined_opiod['relative_opioids'] = summary_joined_opiod['mean']-opiod_mu
print(summary_joined_opiod.head())

          count      mean       std   sum  relative_opioids
practice                                                   
A81005     1507  0.033179  0.179162  50.0         -0.002624
A81007     1454  0.043329  0.203666  63.0          0.007526
A81011     1568  0.046556  0.210753  73.0          0.010753
A81012     1332  0.042793  0.202466  57.0          0.006990
A81017     2150  0.038140  0.191578  82.0          0.002337


How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [41]:
opiod_sigma = joined_opiod['Opiod'].std()
summary_joined_opiod['std_error'] = opiod_sigma/np.sqrt(summary_joined_opiod['count'])
summary_joined_opiod['opioid_scores'] = summary_joined_opiod['relative_opioids']/summary_joined_opiod['std_error']
summary_joined_opiod.head()

,count,mean,std,sum,relative_opioids,std_error,opioid_scores
practice,,,,,,,
A81005,1507,0.033179,0.179162,50.0,-0.002624,0.004786,-0.548306
A81007,1454,0.043329,0.203666,63.0,0.007526,0.004873,1.544557
A81011,1568,0.046556,0.210753,73.0,0.010753,0.004692,2.291795
A81012,1332,0.042793,0.202466,57.0,0.006990,0.005091,1.373060
A81017,2150,0.038140,0.191578,82.0,0.002337,0.004007,0.583168


In [42]:
opioid_scores = summary_joined_opiod[['opioid_scores','count']]
opioid_scores=opioid_scores.sort_values(by='opioid_scores',ascending=False)
print(opioid_scores.head())

          opioid_scores  count
practice                      
Y01852        11.695818      7
Y03006         7.339043      2
Y03668         6.150582     60
G81703         5.123032     36
Y04997         4.958866    321


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [43]:
unique_practices = practices.iloc[:,[0,1]].groupby('code').min()
opioid_scores = opioid_scores.merge(unique_practices, left_index=True,right_index=True,sort=False).reset_index()

print(opioid_scores.head())

# anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

    index  opioid_scores  count                             name
0  Y01852      11.695818      7        NATIONAL ENHANCED SERVICE
1  Y03006       7.339043      2         OUTREACH SERVICE NH / RH
2  Y03668       6.150582     60  BRISDOC HEALTHCARE SERVICES OOH
3  G81703       5.123032     36           H&R P C SPECIAL SCHEME
4  Y04997       4.958866    321                   HMR BARDOC OOH


In [44]:
subset = opioid_scores[['name', 'opioid_scores', 'count']][:100]
tuples = [tuple(x) for x in subset.to_numpy()]
anomalies = tuples

In [45]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [28]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz', compression='gzip')
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [29]:
script_count_16 = scripts16.groupby('bnf_name').agg({'items' : 'count'}).rename(columns={'items' : 'count_16'})
script_count_16_filtered = script_count_16[script_count_16['count_16']>=50]
script_count_16_filtered.head()
# script_count_16_filtered.loc["Butec_Transdermal Patch 5mcg/hr"]

,count_16
bnf_name,
3m Health Care_Cavilon Durable Barrier C,825
3m Health Care_Cavilon No Sting 1ml Barr,231
3m Health Care_Cavilon No Sting 3ml Barr,105
3m Health Care_Cavilon No Sting Barrier,454
A.S Saliva Orthana Spy 50ml (App),84


In [46]:
script_count_17 = joined.groupby('bnf_name').agg({'items' : 'count'}).rename(columns={'items' : 'count_17'})
script_count_17.head()

,count_17
bnf_name,
365 Film 10cm x 12cm VP Adh Film Dress,1
365 Non Adherent 10cm x 10cm Pfa Plas Fa,3
365 Non Adherent 10cm x 20cm Pfa Plas Fa,1
365 Non Woven Island 8cm x 10cm Adh Dres,1
365 Transpt Island 5cm x 7.2cm VP Adh Fi,2


In [48]:
script_growth_all = script_count_17.merge(script_count_16_filtered,left_index=True,right_index=True)
script_growth_all['pct_change']= script_growth_all['count_17']/script_growth_all['count_16'] -1
script_growth_all = script_growth_all.sort_values(by='pct_change',ascending=False).reset_index()
script_growth_required = pd.concat([script_growth_all.head(50), script_growth_all.tail(50)]).reset_index(drop=True)

unique_practices = practices.iloc[:,[0,1]].groupby('code').min()
unique_practices

subset = script_growth_required[['bnf_name', 'pct_change', 'count_16']]
tuples = [tuple(x) for x in subset.to_numpy()]

In [49]:
# script_growth = [("Oxycodone HCl_Tab 30mg M/R", -0.402778, 144)] * 100
# script_growth = [("Oxycodone HCl_Tab 20mg M/R", -0.431818, 181.0)] * 100
script_growth = tuples

In [50]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [51]:
p = 1/len(joined.groupby('bnf_code'))
rates = joined.groupby('bnf_code').agg({'items' : 'count'}).rename(columns={'items' : 'probability'})/ len(joined)
rare_codes = pd.DataFrame(rates[rates['probability']< (0.1*p) ])
rare_codes.loc[:,'is_rare']=1
print(rare_codes.head())

joined_rare_scripts = scripts.merge(rare_codes,how='left',right_index=True,left_on='bnf_code').fillna(0)
joined_rare_scripts['act_cost_nic'] = joined_rare_scripts['is_rare'] * joined_rare_scripts['act_cost']
joined_rare_scripts.head()


           probability  is_rare
bnf_code                       
0101010C0     0.000035        1
0101010F0     0.000002        1
0101010I0     0.000035        1
0101010J0     0.000042        1
0101010N0     0.000023        1


,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,probability,is_rare,act_cost_nic
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0.0,0.0,0.0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0.0,0.0,0.0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0.0,0.0,0.0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0.0,0.0,0.0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0.0,0.0,0.0


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [52]:
total_cost_per_practice = joined_rare_scripts.groupby('practice').sum()[['act_cost','act_cost_nic']]
total_cost_per_practice['proportion'] = total_cost_per_practice['act_cost_nic']  / total_cost_per_practice['act_cost'] 
total_cost_per_practice.head()

,act_cost,act_cost_nic,proportion
practice,,,
A81005,103840.82,1247.83,0.012017
A81007,113482.49,951.06,0.008381
A81011,159507.03,816.02,0.005116
A81012,83296.81,1145.11,0.013747
A81017,232656.17,1712.15,0.007359


Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [53]:
mu_rare =  sum(total_cost_per_practice['act_cost_nic'])/sum(total_cost_per_practice['act_cost'])
relative_rare_cost_prop = total_cost_per_practice[['proportion']] - mu_rare
relative_rare_cost_prop.head()

,proportion
practice,
A81005,-0.003946
A81007,-0.007582
A81011,-0.010847
A81012,-0.002216
A81017,-0.008604


Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [54]:
standard_errors = relative_rare_cost_prop.std()[0]

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [55]:
rare_scores = relative_rare_cost_prop/standard_errors
rare_scores = rare_scores.rename(columns={'proportion' : 'z_scores'})

unique_practices = practices.iloc[:,[0,1]].groupby('code').min()
rare_scores = rare_scores.merge(unique_practices, left_index=True,right_index=True,sort=False)
rare_scores = rare_scores.sort_values(by='z_scores',ascending=False).reset_index()
print(rare_scores.head())

subset = rare_scores[['index', 'name', 'z_scores']].head(100)
tuples = [tuple(x) for x in subset.to_numpy()]

    index   z_scores                               name
0  Y03472  16.262687           CONSULTANT DIABETES TEAM
1  Y05320  15.128648     DMC COMMUNITY DERMATOLOGY RBWF
2  Y04404   7.542139  OUTPATIENTS JUBILEE HEALTH CENTRE
3  Y03484   7.287222   DMC COMMUNITY DERMATOLOGY CLINIC
4  Y04424   6.838614                     DMC HEALTHCARE


In [56]:
# rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100
rare_scripts = tuples
# rare_scripts = [("M85766", "BALSALL HEATH HEALTH CENTRE (S)", rare_scores.iloc[40]['z_scores'])] * 100

In [57]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*